In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import keras
import plotly.express as px
import numpy as np
import tensorflow_model_optimization as tfmot

In [10]:
model = keras.models.load_model('checkpoint/style_model_5.keras', compile=False)
model.trainable = False
model(tf.random.normal((2, 1, 512, 512, 3)))
# model.build((2, 1, 512, 512, 3))
model.summary()

Model: "style_transfer_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ada_in_1 (AdaIN)            multiple                  0         
                                                                 
 encoder_1 (Encoder)         multiple                  3505728   
                                                                 
 decoder_1 (Decoder)         multiple                  3505219   
                                                                 
Total params: 7010955 (26.74 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 7010955 (26.74 MB)
_________________________________________________________________


In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converted = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(converted)

INFO:tensorflow:Assets written to: /tmp/tmpjsuno1_4/assets


INFO:tensorflow:Assets written to: /tmp/tmpjsuno1_4/assets
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 31, Total Ops 135, % non-converted = 22.96 %
 * 31 ARITH ops

- arith.constant:   31 occurrences  (f32: 20, i32: 11)



  (f32: 3)
  (f32: 27)
  (f32: 6)
  (f32: 4)
  (f32: 27)
  (f32: 2)
  (uq_8: 18)
  (f32: 3)
  (f32: 2)
  (f32: 1)
  (f32: 1)
  (f32: 2)
  (f32: 2)
  (f32: 3)


In [33]:
content = '/mnt/s/CV/StyleTransferData/test2015/COCO_test2015_000000011864.jpg'
style = '/mnt/s/CV/StyleTransferData/wikiart_styles/1.jpg'

In [30]:
content = tf.image.resize(tf.image.decode_image(tf.io.read_file(content))[tf.newaxis, ...], (512, 512))
style = tf.image.resize(tf.image.decode_image(tf.io.read_file(style))[tf.newaxis, ...], (512, 512))
input_data = tf.concat((content[tf.newaxis, ...], style[tf.newaxis, ...]), axis=0)

In [7]:
interpreterer = tf.lite.Interpreter('model.tflite')
interpreterer.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
interpreterer.get_input_details()

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([  1,   1, 512, 512,   3], dtype=int32),
  'shape_signature': array([ -1,   1, 512, 512,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]